In [1]:
import numpy as np

In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

tf.executing_eagerly()

True

In [3]:
# thanks https://stackoverflow.com/questions/34199233/how-to-prevent-tensorflow-from-allocating-the-totality-of-a-gpu-memory
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

2021-08-30 10:28:04.283906: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-08-30 10:28:04.307710: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2021-08-30 10:28:04.307750: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: winter
2021-08-30 10:28:04.307761: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: winter
2021-08-30 10:28:04.307867: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.57.2
2021-08-30 10:28:04.307907: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.57.2
2021-08-30 10:28:04.307918: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.57.2


In [4]:
Pa = np.array([[0.71893144, 0.14144765, 0.10698649, 0.03263443],
       [0.06325843, 0.77423272, 0.13928361, 0.02322523],
       [0.02322523, 0.13928361, 0.77423272, 0.06325843],
       [0.03263443, 0.10698649, 0.14144765, 0.71893144]])

Pb = np.array([[0.95154866, 0.01674797, 0.0234783 , 0.00822507],
       [0.03863562, 0.84803839, 0.04732459, 0.0660014 ],
       [0.0660014 , 0.04732459, 0.84803839, 0.03863562],
       [0.00822507, 0.0234783 , 0.01674797, 0.95154866]])

Pc = np.array([[0.88137945, 0.04613594, 0.02686734, 0.04561727],
       [0.05009116, 0.9236776 , 0.0194023 , 0.00682894],
       [0.00682894, 0.0194023 , 0.9236776 , 0.05009116],
       [0.04561727, 0.02686734, 0.04613594, 0.88137945]])
Pm = np.array([[0.81203158, 0.04399917, 0.07848442, 0.06548484],
       [0.08284662, 0.87115553, 0.02024715, 0.0257507 ],
       [0.0257507 , 0.02024715, 0.87115553, 0.08284662],
       [0.06548484, 0.07848442, 0.04399917, 0.81203158]])

pi = np.array([0.14977394, 0.26317362, 0.39165429, 0.19539816])

pi_m = pi @ Pm

J = np.array([[[0.07508613297288518, 0.00466350788803318, 0.0033432901131033796, 0.004042219322757277], [0.002186937387234338, 0.01570096614179908, 0.0011254388766303509, 0.0005015574893929194], [0.002007283482847851, 0.0012535337368050612, 0.013395860029779497, 0.0010220788989917808], [0.0013011086711092986, 0.0016042909684947522, 0.0012404869820179004, 0.011323417297654589]], [[0.03129698648252512, 0.00815953485072002, 0.004394876926278819, 0.0020308813605868174], [0.008288184321098279, 0.14201837079233715, 0.005398759767993272, 0.0017880581744532536], [0.0015277493099248596, 0.00886482820182737, 0.04269081732021271, 0.0028169181161841477], [0.0021015052010260205, 0.011819352252411985, 0.0034103593745217937, 0.02379160092199082]], [[0.02174460265532043, 0.002941403304710777, 0.017006110729978843, 0.0023681733984058486], [0.0022577498981608663, 0.03237071808690972, 0.012426756488990255, 0.0019160961759307586], [0.0022144999652547693, 0.006239396030284971, 0.20953281339937846, 0.012130269098422072], [0.002543170970124578, 0.003973434481032667, 0.011775618913078878, 0.04210892174683528]], [[0.012093125924573445, 0.0012993605630256348, 0.0021464967072447926, 0.0015645368675293004], [0.0010702747292501203, 0.013479197587703494, 0.0016493822206027895, 0.0025896804160072057], [0.0005894417935184289, 0.00127918145334602, 0.022260289874459077, 0.0029430040436876307], [0.005229012301832341, 0.0040480932067737535, 0.006152859243062461, 0.09785943409093556]]])

In [5]:
J_mb = pi_m[:, np.newaxis] * Pb
J_bc = J_mb.T @ Pc
J_ab = Pa.T @ np.diag(pi) @ Pm @ Pb
J_ca = (Pa.T @ np.diag(pi) @ Pm @ Pc).T

assert np.allclose(J_bc, J.sum(axis=0))  # checking I have constructed it correctly
assert np.allclose(J_ab, J.sum(axis=2))
assert np.allclose(J_ca, J.sum(axis=1).T)

J_einsum = np.einsum('i,ij,ik,ku,kv', pi, Pa, Pm, Pb, Pc)
assert np.allclose(J_einsum, J)

In [6]:
# thanks https://github.com/mlgxmez/thelongrun_notebooks/blob/master/MLE_tutorial.ipynb

@tf.function()
def transform_P_matrix(params):
    Pa = tfb.SoftmaxCentered()(params)
    return tf.concat([Pa, Pa[::-1, ::-1]], axis=0)

@tf.function()
def transform(params):
    pi = tfb.SoftmaxCentered()(params[0])
    Pa = transform_P_matrix(params[1:3])
    Pm = transform_P_matrix(params[3:5])
    Pb = transform_P_matrix(params[5:7])
    Pc = transform_P_matrix(params[7:9])
    return pi, Pa, Pm, Pb, Pc
    
@tf.function()
def loss(params, data):
    pi, Pa, Pm, Pb, Pc = transform(params)
    J = tf.einsum('i,ij,ik,ku,kv', pi, Pa, Pm, Pb, Pc)
    loss = tf.reduce_sum(tf.keras.losses.KLDivergence()(J, data))
    return loss

def mle_run(data, loss, parameters, optimizer, tol=1e-9, steps=1000, verbose=False):
    update_list = []
    
    last_loss = None
    for i in range(steps):
        with tf.GradientTape() as tape:
            loss_value = loss(parameters, data)
        gradients = tape.gradient(loss_value, parameters)
        optimizer.apply_gradients([(gradients, parameters)])
    
        if i % 100 == 0:
            update_list.append((
                optimizer.iterations.numpy(),loss_value.numpy(), 
                parameters.numpy()))
            iter_info = f"Step: {optimizer.iterations.numpy()}, initial loss: {loss_value.numpy()}"
            if verbose:
                print(iter_info)
        
        if last_loss and np.abs(last_loss - loss_value) < tol:
            break
        last_loss = loss_value.numpy()
    
    return update_list

In [7]:
%%time
normal_initializer = tf.random_normal_initializer()
params = tf.Variable(normal_initializer(shape=[9, 3], dtype=tf.float32), name='params')

optimizer = tf.optimizers.Adam(0.01)
update_list = mle_run(J, loss, params, optimizer, tol=1e-12, steps=10000, verbose=True)

2021-08-30 10:28:07.455700: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-08-30 10:28:07.477329: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2599990000 Hz
2021-08-30 10:28:07.477837: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa1e8000b60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-08-30 10:28:07.477855: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Step: 1, initial loss: 0.06864514946937561
Step: 101, initial loss: 0.03785013034939766
Step: 201, initial loss: 0.019931580871343613
Step: 301, initial loss: 0.007992316037416458
Step: 401, initial loss: 0.0012685999972745776
Step: 501, initial loss: 0.0005335374153219163
Step: 601, initial loss: 0.00031813705572858453
Step: 701, initial loss: 0.00020524259889498353
Step: 801, initial loss: 0.0001373789709759876
Step: 901, initial loss: 0.00010011563426814973
Step: 1001, initial loss: 7.830371032468975e-05
Step: 1101, initial loss: 6.327212031465024e-05
Step: 1201, initial loss: 5.123576556798071e-05
Step: 1301, initial loss: 4.03955637011677e-05
Step: 1401, initial loss: 3.020931762875989e-05
Step: 1501, initial loss: 2.1380270482040942e-05
Step: 1601, initial loss: 1.5132383850868791e-05
Step: 1701, initial loss: 1.1319374607410282e-05
Step: 1801, initial loss: 8.942486601881683e-06
Step: 1901, initial loss: 7.2859766078181565e-06
Step: 2001, initial loss: 6.075410055927932e-06
Step

In [8]:
for p, phat in zip([pi, Pa, Pm, Pb, Pc], transform(params)):
    print(p)
    print(phat.numpy())
    print()

[0.14977394 0.26317362 0.39165429 0.19539816]
[0.4265618  0.12358643 0.16080192 0.28904986]

[[0.71893144 0.14144765 0.10698649 0.03263443]
 [0.06325843 0.77423272 0.13928361 0.02322523]
 [0.02322523 0.13928361 0.77423272 0.06325843]
 [0.03263443 0.10698649 0.14144765 0.71893144]]
[[0.02285772 0.14069095 0.7598294  0.07662188]
 [0.82079244 0.04331525 0.0955871  0.04030516]
 [0.04030516 0.0955871  0.04331525 0.82079244]
 [0.07662188 0.7598294  0.14069095 0.02285772]]

[[0.81203158 0.04399917 0.07848442 0.06548484]
 [0.08284662 0.87115553 0.02024715 0.0257507 ]
 [0.0257507  0.02024715 0.87115553 0.08284662]
 [0.06548484 0.07848442 0.04399917 0.81203158]]
[[0.03101633 0.8166028  0.02154983 0.13083106]
 [0.8468377  0.08314174 0.00906565 0.06095482]
 [0.06095482 0.00906565 0.08314174 0.8468377 ]
 [0.13083106 0.02154983 0.8166028  0.03101633]]

[[0.95154866 0.01674797 0.0234783  0.00822507]
 [0.03863562 0.84803839 0.04732459 0.0660014 ]
 [0.0660014  0.04732459 0.84803839 0.03863562]
 [0.0082